In [1]:
import h5py
import numpy as np
import time
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
import plotly.offline as pyo

# Part 1: Exploring the intraday data

In [2]:
def get_intradata_and_ticker(source):
    try:
        f = h5py.File(source,'r')
    except OSError:
        print('OSError')
        return [], []
    df = f['df']
    values = np.concatenate(
    (
        df['axis1'][:].reshape(df['block4_values'].shape[0], 1),
        df['block0_values'][:],
        df['block1_values'][:]
    ), 
    axis=1)
    columns = np.concatenate(
    (
        ['axis1'],
        df['block0_items'][:],
        df['block1_items'][:]
    ), 
    axis=0)
    intraday = pd.DataFrame(data=values, columns=columns)
    ticker = pd.DataFrame(data = df['block2_values'][0], columns = ['ticker'])
    return intraday, ticker  
    

In [3]:
intraday, ticker = get_intradata_and_ticker('intraday_data/2021-08-18.hd5')

In [4]:
times = []
for el in intraday.time.values:
    times.append(datetime.datetime(*list(time.localtime(el)[:-2])))
intraday.time = times

In [5]:
intraday.head(3)

axis1  open  high  close   low  volume                       time
0    0.0  24.0  24.0   24.0  24.0   100.0 2021-08-18 18:09:00.000002
1    1.0  24.0   NaN   24.0   NaN     NaN 2021-08-18 18:10:00.000002
2    2.0  24.0   NaN   24.0   NaN     NaN 2021-08-18 18:11:00.000002

In [6]:
indices_of_zero_time = intraday.axis1[intraday.axis1==0.0].index

In [7]:
len(indices_of_zero_time) # number of tickers collected during the day

863

In [8]:
pyo.init_notebook_mode()

In [9]:
#let's plot some OHLC graphs with Plotly

for i in range(len(indices_of_zero_time))[5:8]: 
    x = intraday.time[indices_of_zero_time[i]:indices_of_zero_time[i+1]]
    close_ = intraday.close[indices_of_zero_time[i]:indices_of_zero_time[i+1]]
    open_ = intraday.open[indices_of_zero_time[i]:indices_of_zero_time[i+1]]
    high_ = intraday.high[indices_of_zero_time[i]:indices_of_zero_time[i+1]]
    low_ = intraday.low[indices_of_zero_time[i]:indices_of_zero_time[i+1]]

    fig = go.Figure(data=go.Ohlc(x=x,
                        open=open_,
                        high=high_,
                        low=low_,
                        close=close_)
                   )
    fig.update_layout(
        title=f'Ticker №{i+1}'
    )
    fig.show()

## Summary:
- Intraday data files consists of number of tickers and corresponding data about them durind the day (e.g. open, high, low and close prices during every minute)
- In average, every day is 390 minutes (= 6,5 hours) long. But some of the days can be either longer or shorter. Why?
- A huge question is how to determine the exact ticker name??? Intraday file has 'ticker' property which is a list with one high-dimensional array inside. But I can't determine how to retrieve the names of tickers from this array. It is possible to determine a number of these tickers from 'axis1' property, but this number don't correspond with the length of the array. I have an assumption, that numbers in this array may be linked with RGB pallete, beacause numbers in the array are in range [0, 256] if I am not mistaken. Thus, this needs a further investigation...

# Part 2: Scenario review
    Here I'm going to consider a situation, when we want to enter a position with news with for which the model predicted a high score.

In [10]:
tpme = pd.read_hdf('ticker_price_moment_exit.hd5')

From this database we can extract intraday data of a certain tickers at the moment of the news release.

In [11]:
tpme.head(3)

ticker bidPrice bidSize askPrice askSize  lastUpdated lastSalePrice  \
0     VS        0       0        0       0  1.62932e+12          4.33   
1  VSSYW        0       0        0       0  1.62819e+12          1.55   
3   BTBT        0       0        0       0  1.63251e+12          9.44   

  lastSaleSize   lastSaleTime volume current_price   timestamp  \
0            6  1629316790023     46          4.24  1629716406   
1           25  1628178157384     25          2.08  1629716406   
3            1  1632513582057  38333          9.46  1632747622   

                            news_id  
0  0000d467d1a52a6a84fc8b30adbe8ec2  
1  0000d467d1a52a6a84fc8b30adbe8ec2  
3  0007ea6c22fc24e1f34021185ad07e4e

In [12]:
scores = pd.read_hdf('scores_retrotesting.hd5')

In [13]:
scores.head(3)

observation_id                           news_id  10_True_0.05 ticker  \
0               3  000fb4a135119a9cf49fe51f91e37add           0.0   SALM   
1               4  000fda8be4442b6ffdcc247693075ffa           0.0    PCT   
2              13  00183e71753b66ed27eb5a59886eed7d           0.0    RXT   

   final_score  final_probabilities  \
0    -1.322091             0.210471   
1    -1.206645             0.230295   
2    -4.440871             0.011648   

   probability_of_conversion_calibrated_by_source.name  probability  
0                                           0.052152       0.043633  
1                                           0.000232       0.045608  
2                                           0.000001       0.012884

From this database we can extract scores and probabilities that target '10_True_0.05' equals 1 (ticker price is going to have a 5% change in the next 10 minutes)

Let's sort by descending 'final_score' and take top-20

In [14]:
scores.sort_values(by='final_score', ascending=False, inplace=True)

In [15]:
best_scores = scores.nlargest(20, 'final_score')

In [16]:
best_scores.head(3)

observation_id                           news_id  10_True_0.05 ticker  \
9784            31732  df80c5edd0782c030a7f05ba8bf8f9f4           0.0   AVNW   
2064             2913  144dd04df5978a4844fdd3eb55e2f24b           0.0   MNTK   
17624           12215  55cfeb5872b8d6cd480c778447fe1b2f           1.0   BASE   

       final_score  final_probabilities  \
9784      5.159716             0.994289   
2064      4.420361             0.988113   
17624     4.363404             0.987425   

       probability_of_conversion_calibrated_by_source.name  probability  
9784                                            0.376994       0.380742  
2064                                            0.325499       0.313657  
17624                                           0.316157       0.308758

Now we can extract pairs [ticker, news_id] and find them in 'ticker_price_moment_exit.hd5'

In [17]:
high_score_positions = tpme[tpme.news_id.isin(best_scores.news_id)&
                            tpme.ticker.isin(best_scores.ticker)]

### These are the positions with news for which the model predicted a high score

In [18]:
high_score_positions

ticker bidPrice bidSize askPrice askSize    lastUpdated lastSalePrice  \
8318    SWBI        0       0        0       0    1.63053e+12         24.87   
10217   MRVL        0       0        0       0  1630008113675          63.2   
16239   COCP        0       0        0       0    1.63346e+12         0.973   
18505   BASE        0       0        0       0    1.63113e+12         51.67   
33127   PRQR        0       0        0       0  1631131213531          6.81   
41822   TCRR        0       0        0       0  1630526400549         17.56   
42389   PSTV        0       0        0       0    1.63295e+12          1.91   
45990   RXST        0       0        0       0  1631217601276         16.22   
47747   AVNW        0       0        0       0  1629921600543          38.7   
49108   FARM        0       0        0       0  1631217450724         7.955   

      lastSaleSize   lastSaleTime  volume current_price   timestamp  \
8318            20  1630526395788   95675         24.88  1630526713   
10217          178  1630007995031  193761         63.24  1630008322   
16239          100  1633463030106     400        0.9701  1633524545   
18505           67  1631131193134   29952         51.58  1631131515   
33127           20  1631131197961    6172          6.81  1631132103   
41822            5  1630526398679   14524         17.54  1630588747   
42389          200  1632945516708    1600           1.9  1633003204   
45990           18  1631217588675   10022         15.99  1631217907   
47747          100  1629921501793    8745         38.72  1629921792   
49108          100  1631217424364    7957          7.94  1631217607   

                                news_id  
8318   25d601176eea9c0fdcc14e8313280889  
10217  2ef6d6cd7f86d133fc9a29e79a18bc8b  
16239  4b5d4ab87b48f0477ce5d4b25a17bf3f  
18505  55cfeb5872b8d6cd480c778447fe1b2f  
33127  9a3016a2eddd4c836dea4fd09c5e410b  
41822  c3f594fb67005de238b803564880a56f  
42389  c66c203401eff49d1d0ccd3446918c59  
45990  d720e751340714dada5f1877db4cae04  
47747  df80c5edd0782c030a7f05ba8bf8f9f4  
49108  e5cd210a1d4c2aa6b28146548cbb383b